# Gráfica análisis de signatura operador Teukolsky.

In [24]:
import numpy as np
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, FloatSlider
    widgets_available = True
except Exception:
    widgets_available = False

def Delta(r, M, a):
    return r**2 - 2*M*r + a**2

def compute_detT(r, M=1.0, a=0.7, theta=np.pi/2):
    m_mass = M
    sin_theta_sq = np.sin(theta)**2
    d = Delta(r, M, a)
    A = ((r**2 + a**2)**2)/d - a**2*sin_theta_sq
    D = (a**2)/d - 1.0/sin_theta_sq
    C = (2*m_mass*a*r)/d
    return A*D - C**2

def plot_detT_with_delta(M=1.0, a=0.7, theta=np.pi/2):
    disc = M**2 - a**2  # discriminante
    has_horizons = disc >= 0  # soluciones reales
    if has_horizons:  # calculo de los  horizontes
        r_plus = M + np.sqrt(disc)
        r_minus = M - np.sqrt(disc)
    else:
        r_plus = r_minus = None  # se descarta las  soluciones imaginarias

    r = np.linspace(0.05, 4.0, 6000)
    # quitar valores cercanos a los  horizontes pues diverge.
    mask = np.ones_like(r, dtype=bool)
    if has_horizons:
        mask &= (np.abs(r - r_plus) > 1e-4)
        mask &= (np.abs(r - r_minus) > 1e-4)
    r_safe = r[mask]

    det_vals = compute_detT(r_safe, M=M, a=a, theta=theta)
    delta_vals = Delta(r_safe, M, a)



    fig, ax1 = plt.subplots(figsize=(8,4.5))
    ax1.plot(r_safe, det_vals, label=rf"det($\mathcal{{L}}_{{t \phi}}$)")
    ax1.axhline(0, linestyle='--')
    if has_horizons:
        ax1.axvline(r_minus, linestyle=':', alpha=0.7, color="red", label=rf"$r_{{-}}$ , $r_{{+}}$")
        ax1.axvline(r_plus, linestyle=':', alpha=0.7, color="red")

    ax1.set_yscale('symlog', linthresh=1e-3)
    ax1.set_xlabel('r')
    ax1.set_ylabel(rf'det($\mathcal{{L}}_{{t \phi}}$)')
    title= rf"Signatura($\mathcal{{L}}$) vs $r$"
    # title = rf"det($\mathcal{{L}}_{{t \phi}}$),sign($\mathcal{{B}}$) vs r"
    title += f"\n(m={M:.2f}, a={a:.2f}, θ={theta:.2f} rad)"
    ax1.set_title(title)
    ax1.legend(loc='upper right', fontsize=8)

    # Second y-axis for sign(Δ)
    ax2 = ax1.twinx()
    ax2.plot(r_safe, delta_vals, linestyle='-', alpha=0.6, label=rf"sign($\mathcal{{B}}$)",color="green")
    ax2.set_ylabel(r'$sign(\mathcal{B})$')
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_yticks([-1, 0, 1])
    # Merge legends
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper right', fontsize=8)

    fig.tight_layout()

if widgets_available:
    _ = interact(
        plot_detT_with_delta,
        M=FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='M (m)'),
        a=FloatSlider(value=0.7, min=0.0, max=2.5, step=0.05, description='a (spin)'),
        theta=FloatSlider(value=np.pi/2, min=1e-3, max=np.pi-1e-3, step=0.01, description='θ (rad)')
    )
else:
    plot_detT_with_delta(M=1.0, a=0.7, theta=np.pi/2)


interactive(children=(FloatSlider(value=1.0, description='M (m)', max=3.0, min=0.5), FloatSlider(value=0.7, de…